In [1]:
import importlib_resources as resources
import sys
import glob
import pandas as pd
import threading
from datetime import datetime, timedelta
import os

#from etherscan import Etherscan
import json
import time
import requests
from hexbytes import HexBytes
#from web3 import Web3
import flatdict
from datetime import datetime
from tqdm import tqdm

pd.set_option('display.max_columns', None)

import mysql.connector

# 1. Abonrmal Wallets

## 1.1 Load Abnormal Wallets, Trx and ERC-20:

### 1.1.1 Abnormal Wallets

In [2]:
import glob
import pandas as pd

df_Abn_wllts = pd.read_csv('./BlockedListed - Dune.csv')
df_Abn_wllts['datetime'] = pd.to_datetime(df_Abn_wllts['datetime'])
df_Abn_wllts.head()

,wallet_hash,datetime
0,0x87b220c084466dd429f9fc767f53592f35ec94bb,2022-11-25 16:08:00
1,0x200f8f5135b90d75d54ecbe34367343f77867096,2022-11-25 02:25:00
2,0xe1c63ef1a2f00840cba26d13249a247bc8ad40bf,2022-11-04 22:18:00
3,0x7a0c5bf3ac8778bd63efb914f89c048447331dab,2022-11-04 22:17:00
4,0xe4af40b5b6a4c7ba9aacfe86a95f3da924648ac2,2022-11-04 22:17:00


In [3]:
len(df_Abn_wllts)

250

### 1.1.2 Abnrnml Trx

In [4]:
df_Abn_trx = pd.read_csv('./agg_fraud_trx.csv')

### 1.1.3 Abnrml ERC-20

In [5]:
df_Abn_erc20_trnsf_appr = pd.concat(map(pd.read_csv, glob.glob('./02.Data/01.Abnromal/03.ERC-20/ERC20_*.csv')))
df_Abn_erc20_trnsf_appr.head()

,Unnamed: 0,hash,blockNumber,timestamp,receiverIsContract,sender,receiver,ERC20Info_eventInfo_type,ERC20Info_eventInfo_addressApproved,ERC20Info_eventInfo_tokenAllowance,ERC20Info_eventInfo_approvedSpenderIsContract,ERC20Info_tokenAddress,ERC20Info_tokenDetails_totalSupply,ERC20Info_tokenDetails_symbol,ERC20Info_tokenDetails_name,ERC20Info_tokenDetails_decimals,ethValue,ERC20Info_eventInfo_transferReceiver,ERC20Info_eventInfo_tokenQuantity,ERC20Info_eventInfo_transferReceiverIsContract
0,0,0x7c66dab17478ed7c6b298c4d2eb32e32b77f8911e01d...,15307582.0,1.660042e+09,True,0x35663b9a8e4563eefdf852018548b4947b20fce6,0xa11bd36801d8fa4448f0ac4ea7a62e3634ce8c7c,approval,0x35663b9a8e4563eefdf852018548b4947b20fce6,1157920892373161954235709850086879078532699846...,True,0xa11bd36801d8fa4448f0ac4ea7a62e3634ce8c7c,100000000000000000000000000,abr,allbridge,18.0,0.0,NaN,NaN,NaN
1,0,0xf928f23c0d569c734a118840cf61156d38dc8198b01c...,15307545.0,1.660042e+09,True,0x35663b9a8e4563eefdf852018548b4947b20fce6,0x3845badade8e6dff049820680d1f14bd3903a5d0,approval,0x35663b9a8e4563eefdf852018548b4947b20fce6,1157920892373161954235709850086879078532699846...,True,0x3845badade8e6dff049820680d1f14bd3903a5d0,3000000000000000000000000000,sand,sand,18.0,0.0,NaN,NaN,NaN
2,0,0x76160c5ea56098ede9d9e468b02cd8dfce2270f1c183...,15307535.0,1.660042e+09,True,0x35663b9a8e4563eefdf852018548b4947b20fce6,0xdac17f958d2ee523a2206206994597c13d831ec7,approval,0x35663b9a8e4563eefdf852018548b4947b20fce6,1157920892373161954235709850086879078532699846...,True,0xdac17f958d2ee523a2206206994597c13d831ec7,32297366521996886,usdt,tether usd,6.0,0.0,NaN,NaN,NaN
3,0,0x2b0ae39332ba9d476609ccab0e44791e80490dc2d511...,15307504.0,1.660041e+09,True,0x35663b9a8e4563eefdf852018548b4947b20fce6,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,approval,0x35663b9a8e4563eefdf852018548b4947b20fce6,1157920892373161954235709850086879078532699846...,True,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,10000000000000000000000000000,matic,matic token,18.0,0.0,NaN,NaN,NaN
4,0,0x5d706fd09a1c6a00ac50a50c33d859856958827acc31...,13715896.0,1.638291e+09,True,0x35663b9a8e4563eefdf852018548b4947b20fce6,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,approval,0x35663b9a8e4563eefdf852018548b4947b20fce6,1157920892373161954235709850086879078532699846...,True,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,10000000000000000000000000000,matic,matic token,18.0,0.0,NaN,NaN,NaN


In [6]:
df_Abn_erc20_trnsf_appr.describe()

,blockNumber,timestamp,ERC20Info_tokenDetails_decimals,ethValue
count,5.711500e+04,5.711500e+04,57115.000000,5.711500e+04
mean,1.169086e+07,1.609949e+09,17.537512,5.252561e-08
std,3.395380e+06,4.810674e+07,2.309463,1.255296e-05
min,3.987379e+06,1.499420e+09,0.000000,0.000000e+00
25%,1.213927e+07,1.617089e+09,18.000000,0.000000e+00
50%,1.298775e+07,1.628470e+09,18.000000,0.000000e+00
75%,1.358888e+07,1.636553e+09,18.000000,0.000000e+00
max,1.613385e+07,1.670428e+09,18.000000,3.000000e-03


In [7]:
df_Abn_erc20_other = pd.concat(map(pd.read_csv, glob.glob('./02.Data/01.Abnromal/03.ERC-20/OTHER_*.csv')))

In [8]:
df_Abn_erc20_other = df_Abn_erc20_other.rename(columns={'ERC20Info_eventInfo':'ERC20Info_eventInfo_type'}).fillna('other')

In [9]:
df_abnrm_erc20 = df_Abn_erc20_trnsf_appr.append(df_Abn_erc20_other, ignore_index=True)

/tmp/ipykernel_10250/2678407774.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_abnrm_erc20 = df_Abn_erc20_trnsf_appr.append(df_Abn_erc20_other, ignore_index=True)


In [10]:
df_abnrm_erc20.shape

(59112, 20)

In [11]:
df_abnrm_erc20.head()

,Unnamed: 0,hash,blockNumber,timestamp,receiverIsContract,sender,receiver,ERC20Info_eventInfo_type,ERC20Info_eventInfo_addressApproved,ERC20Info_eventInfo_tokenAllowance,ERC20Info_eventInfo_approvedSpenderIsContract,ERC20Info_tokenAddress,ERC20Info_tokenDetails_totalSupply,ERC20Info_tokenDetails_symbol,ERC20Info_tokenDetails_name,ERC20Info_tokenDetails_decimals,ethValue,ERC20Info_eventInfo_transferReceiver,ERC20Info_eventInfo_tokenQuantity,ERC20Info_eventInfo_transferReceiverIsContract
0,0,0x7c66dab17478ed7c6b298c4d2eb32e32b77f8911e01d...,15307582.0,1.660042e+09,True,0x35663b9a8e4563eefdf852018548b4947b20fce6,0xa11bd36801d8fa4448f0ac4ea7a62e3634ce8c7c,approval,0x35663b9a8e4563eefdf852018548b4947b20fce6,1157920892373161954235709850086879078532699846...,True,0xa11bd36801d8fa4448f0ac4ea7a62e3634ce8c7c,100000000000000000000000000,abr,allbridge,18.0,0.0,NaN,NaN,NaN
1,0,0xf928f23c0d569c734a118840cf61156d38dc8198b01c...,15307545.0,1.660042e+09,True,0x35663b9a8e4563eefdf852018548b4947b20fce6,0x3845badade8e6dff049820680d1f14bd3903a5d0,approval,0x35663b9a8e4563eefdf852018548b4947b20fce6,1157920892373161954235709850086879078532699846...,True,0x3845badade8e6dff049820680d1f14bd3903a5d0,3000000000000000000000000000,sand,sand,18.0,0.0,NaN,NaN,NaN
2,0,0x76160c5ea56098ede9d9e468b02cd8dfce2270f1c183...,15307535.0,1.660042e+09,True,0x35663b9a8e4563eefdf852018548b4947b20fce6,0xdac17f958d2ee523a2206206994597c13d831ec7,approval,0x35663b9a8e4563eefdf852018548b4947b20fce6,1157920892373161954235709850086879078532699846...,True,0xdac17f958d2ee523a2206206994597c13d831ec7,32297366521996886,usdt,tether usd,6.0,0.0,NaN,NaN,NaN
3,0,0x2b0ae39332ba9d476609ccab0e44791e80490dc2d511...,15307504.0,1.660041e+09,True,0x35663b9a8e4563eefdf852018548b4947b20fce6,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,approval,0x35663b9a8e4563eefdf852018548b4947b20fce6,1157920892373161954235709850086879078532699846...,True,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,10000000000000000000000000000,matic,matic token,18.0,0.0,NaN,NaN,NaN
4,0,0x5d706fd09a1c6a00ac50a50c33d859856958827acc31...,13715896.0,1.638291e+09,True,0x35663b9a8e4563eefdf852018548b4947b20fce6,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,approval,0x35663b9a8e4563eefdf852018548b4947b20fce6,1157920892373161954235709850086879078532699846...,True,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,10000000000000000000000000000,matic,matic token,18.0,0.0,NaN,NaN,NaN


In [12]:
#timestamp to datetime
from datetime import datetime

df_abnrm_erc20['TrxDatetime'] = [datetime.fromtimestamp(x) for x in df_abnrm_erc20['timestamp']]

## 1.2 Generate Eth Transactional Features

In [13]:
df_Abn_trx['Trx_Datetime'] = pd.to_datetime(df_Abn_trx['Trx_Datetime'])

In [14]:
df_abn_out_trx = pd.DataFrame(columns=['received:minValue', 'received:maxValue', 'received:avgValue',
       'sent:minValue', 'sent:maxValue', 'sent:avgValue',
       'sentToContract:minValue', 'sentToContract:maxValue',
       'sentToContract:avgValue', 'total:transactions', 'total:etherSent',
       'total:etherReceived', 'total:etherSentToContract', 'total:sentTxn',
       'total:receivedTxn', 'total:sentToContractTxn',
       'total:uniqueReceivedFromAddresses', 'total:uniqueSentToAddresses',
       'time:avgBetweenSentTxn', 'time:avgBetweenReceivedTxn',
       'time:avgBetweenTxn', 'time:diffBetweenFirstAndLastSent',
       'time:diffBetweenFirstAndLastReceived',
       'time:diffBetweenFirstAndLastTxn', 'walletAddress', 'currentTime'])

In [15]:
from pandas.tseries.offsets import DateOffset
from eth_feature_functions import get_processed_erc20_txn_list_analytics
from eth_feature_functions import get_processed_eth_txn_list_analytics
from eth_feature_functions import unflatten
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def dateoffset3Months(df, walletAddress):
    
    detected_date = df[df['wallet_hash']==walletAddress]['datetime'].values[0]
    min_date = detected_date - DateOffset(months=3)
    
    return detected_date, min_date


# Create a list out of all the transaction hashes of transactions to and from the walletAddress
with tqdm(total=len(df_Abn_wllts.wallet_hash)) as pbar:
    for walletAddress in list(df_Abn_wllts.wallet_hash):
        
        detected_date, min_date = dateoffset3Months(df_Abn_wllts, walletAddress)

        df = df_Abn_trx[((df_Abn_trx['Trx_From_Wallet'].str.lower()==walletAddress) 
                        | (df_Abn_trx['Trx_To_Wallet'].str.lower()==walletAddress))
                        & (df_Abn_trx['Trx_Datetime'] >= min_date)
                        & (df_Abn_trx['Trx_Datetime'] <= detected_date)]

        pbar.update(1)

        df = df.rename(columns = lambda x: x.replace('_', ''))

        flattenedDictList = df.values.tolist()
        ethList = []

        for values in flattenedDictList:
            d = dict(zip(df.keys(), values))
            ethList.append(unflatten(d, '_'))

        analytics = flatdict.FlatDict(get_processed_eth_txn_list_analytics(ethList, walletAddress))

        # Transform all values to lower case
        for key in analytics:
            if type(analytics[key]) == str:
                analytics[key] = analytics[key].lower()
            analytics[key] = [analytics[key]]

        df_abn_out_trx = df_abn_out_trx.append(pd.DataFrame(dict(analytics)), ignore_index=True)
        

100%|██████████| 250/250 [00:12<00:00, 19.47it/s]


In [16]:
#df_abn_out_trx = df_abn_out_trx.drop(columns=['currentTime'])
df_abn_out_trx.head()

,received:minValue,received:maxValue,received:avgValue,sent:minValue,sent:maxValue,sent:avgValue,sentToContract:minValue,sentToContract:maxValue,sentToContract:avgValue,total:transactions,total:etherSent,total:etherReceived,total:etherSentToContract,total:sentTxn,total:receivedTxn,total:sentToContractTxn,total:uniqueReceivedFromAddresses,total:uniqueSentToAddresses,time:avgBetweenSentTxn,time:avgBetweenReceivedTxn,time:avgBetweenTxn,time:diffBetweenFirstAndLastSent,time:diffBetweenFirstAndLastReceived,time:diffBetweenFirstAndLastTxn,walletAddress,currentTime
0,4.325361e-04,2.233454,0.072694,0.0,2.200000,0.033311,0.0,0.0,0.0,143,2.997991,3.852804,0.0,90,53,79,50,38,0 days 05:17:59.600000,0 days 09:47:19.018867924,0 days 03:20:08.139860139,19 days 20:59:24,21 days 14:47:48,19 days 20:59:24,0x87b220c084466dd429f9fc767f53592f35ec94bb,NaN
1,1.800000e-08,9.488508,2.372131,NaN,NaN,NaN,0.0,0.0,0.0,4,0.000000,9.488526,0.0,0,4,0,3,0,NaT,0 days 21:49:30,0 days 21:49:30,NaT,3 days 15:18:00,3 days 15:18:00,0x200f8f5135b90d75d54ecbe34367343f77867096,NaN
2,1.000000e-03,0.001000,0.001000,0.0,0.000000,0.000000,0.0,0.0,0.0,2,0.000000,0.001000,0.0,1,1,1,1,1,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0xe1c63ef1a2f00840cba26d13249a247bc8ad40bf,NaN
3,6.358557e-04,0.012194,0.003856,0.0,0.001724,0.000345,0.0,0.0,0.0,14,0.003447,0.015424,0.0,10,4,8,3,2,0 days 01:40:16.800000,0 days 04:57:57,0 days 01:11:37.714285714,0 days 16:42:48,0 days 19:51:48,0 days 16:42:48,0x7a0c5bf3ac8778bd63efb914f89c048447331dab,NaN
4,1.238191e-02,0.012382,0.012382,0.0,0.000000,0.000000,0.0,0.0,0.0,3,0.000000,0.024764,0.0,1,2,1,1,1,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0xe4af40b5b6a4c7ba9aacfe86a95f3da924648ac2,NaN


In [17]:
df_abn_out_trx.shape

(88, 26)

## 1.3 Generate ERC-20 Features

In [18]:
#datetime for timestamp int type:
df_abnrm_erc20['timestamp'] = pd.to_datetime(df_abnrm_erc20['timestamp'], unit='s')

In [19]:

def process_erc20_analytics(wllt_list, df, erc20_type):
    
    df_out = pd.DataFrame(columns=['received:minValue', 'received:maxValue', 'received:avgValue',
       'sent:minValue', 'sent:maxValue', 'sent:avgValue', 'sentToContract:minValue', 'sentToContract:maxValue',
       'sentToContract:avgValue', 'total:transactions', 'total:etherSent', 'total:etherReceived', 'total:etherSentToContract', 'total:sentTxn',
       'total:receivedTxn', 'total:sentToContractTxn', 'total:uniqueReceivedFromAddresses', 'total:uniqueSentToAddresses',
       'time:avgBetweenSentTxn', 'time:avgBetweenReceivedTxn', 'time:avgBetweenTxn', 'time:diffBetweenFirstAndLastSent',
       'time:diffBetweenFirstAndLastReceived', 'time:diffBetweenFirstAndLastTxn', 'erc20:uniqueTokenAddressesSent',
       'erc20:uniqueTokenAddressesReceived', 'erc20:mostSentTokenAddress', 'erc20:mostReceivedTokenAddress', 'erc20:uniqueUnknownValueTokens',
       'erc20:transactionsWithUnknownValueTokens', 'walletAddress','currentTime'])
    
    with tqdm(total=len(wllt_list)) as pbar:
        for walletAddress in wllt_list:

            pbar.update(1)
            df_ = df[((df['sender'].str.lower()==walletAddress) 
                                | (df['receiver'].str.lower()==walletAddress))
                               & (df['ERC20Info_eventInfo_type']==erc20_type)]
            
            flattenedDictList = df_.values.tolist()
            ethList = []

            for values in flattenedDictList:
                d = dict(zip(df_.keys(), values))
                ethList.append(unflatten(d, '_'))

            analytics = flatdict.FlatDict(get_processed_erc20_txn_list_analytics(ethList, walletAddress))

            # Transform all values to lower case
            for key in analytics:
                if type(analytics[key]) == str:
                    analytics[key] = analytics[key].lower()
                analytics[key] = [analytics[key]]

            # Must be converted to dict from FlatDict
            df_out = df_out.append(pd.DataFrame(dict(analytics)), ignore_index=True)
            
    return df_out

### 1.3.1 Approvals

In [20]:
erc20_type = 'approval'
df_abn_out_erc20_app = process_erc20_analytics(list(df_Abn_wllts.wallet_hash), df_abnrm_erc20, erc20_type)

100%|██████████| 250/250 [00:10<00:00, 24.64it/s]


In [21]:
df_abn_out_erc20_app = df_abn_out_erc20_app.drop(columns=['received:minValue','received:maxValue','received:avgValue',
                                                              'sent:minValue','sent:maxValue','sent:avgValue', 
                                                              'sentToContract:minValue','sentToContract:maxValue',
                                                              'sentToContract:avgValue', 'time:avgBetweenSentTxn',
                                                          'time:avgBetweenReceivedTxn', 'time:diffBetweenFirstAndLastSent',
                                                          'time:diffBetweenFirstAndLastReceived', 'erc20:mostReceivedTokenAddress',
                                                         'currentTime'])


In [22]:
df_abn_out_erc20_app.columns = df_abn_out_erc20_app.columns.str.replace(":", "_", regex=True)

In [23]:
df_abn_out_erc20_app = df_abn_out_erc20_app.rename(columns={c: c+'_approval' 
                                                            for c in df_abn_out_erc20_app.columns 
                                                                if c not in ['walletAddress','currentTime']})

### 1.3.2 Transfers

In [24]:
erc20_type = 'transfer'
df_abn_out_erc20_transfer = process_erc20_analytics(list(df_Abn_wllts.wallet_hash), df_abnrm_erc20, erc20_type)

100%|██████████| 250/250 [00:11<00:00, 22.37it/s]


In [25]:
df_abn_out_erc20_transfer = df_abn_out_erc20_transfer.drop(columns=['received:minValue','received:maxValue','received:avgValue',
                                                              'sent:minValue','sent:maxValue','sent:avgValue', 
                                                              'sentToContract:minValue','sentToContract:maxValue',
                                                              'sentToContract:avgValue', 'time:avgBetweenSentTxn',
                                                          'time:avgBetweenReceivedTxn', 'time:diffBetweenFirstAndLastSent',
                                                          'time:diffBetweenFirstAndLastReceived', 'erc20:mostReceivedTokenAddress',
                                                                   'currentTime'])


In [26]:
df_abn_out_erc20_transfer.columns = df_abn_out_erc20_transfer.columns.str.replace(":", "_", regex=True)

In [27]:
df_abn_out_erc20_transfer = df_abn_out_erc20_transfer.rename(columns={c: c+'_transfer' 
                                                            for c in df_abn_out_erc20_transfer.columns 
                                                                if c not in ['walletAddress','currentTime']})

### 1.3.3 Other

In [28]:
erc20_type = 'other'
df_abn_out_erc20_other = process_erc20_analytics(list(df_Abn_wllts.wallet_hash), df_abnrm_erc20, erc20_type)

100%|██████████| 250/250 [00:08<00:00, 28.83it/s]


In [29]:
df_abn_out_erc20_other = df_abn_out_erc20_other.drop(columns=['received:minValue','received:maxValue','received:avgValue',
                                                              'sent:minValue','sent:maxValue','sent:avgValue', 
                                                              'sentToContract:minValue','sentToContract:maxValue',
                                                              'sentToContract:avgValue', 'time:avgBetweenSentTxn',
                                                              'time:diffBetweenFirstAndLastReceived', 'total:etherSent',
                                                              'total:etherReceived','total:etherSentToContract',
                                                              'total:sentTxn','total:receivedTxn','total:sentToContractTxn',
                                                             'currentTime'])


In [30]:
df_abn_out_erc20_other.columns = df_abn_out_erc20_other.columns.str.replace(":", "_", regex=True)

In [31]:
df_abn_out_erc20_other = df_abn_out_erc20_other.rename(columns={c: c+'_other' 
                                                            for c in df_abn_out_erc20_other.columns 
                                                                if c not in ['walletAddress','currentTime']})

## 1.3 Agg erc-20 analytics

In [32]:
erc20_abn_analytics = df_abn_out_erc20_other.merge(df_abn_out_erc20_transfer,
                                                   left_on='walletAddress',
                                                   right_on='walletAddress', how='outer')
erc20_abn_analytics = erc20_abn_analytics.merge(df_abn_out_erc20_app,
                                                left_on='walletAddress',
                                                right_on='walletAddress', how='outer')

## 1.4 Agg All Analytics create Target and Save!

In [33]:
erc20_abn_analytics = erc20_abn_analytics.merge(df_abn_out_trx, 
                                                left_on='walletAddress',
                                                right_on='walletAddress', how='outer')

In [34]:
erc20_abn_analytics.head()

,total_transactions_other,total_uniqueReceivedFromAddresses_other,total_uniqueSentToAddresses_other,time_avgBetweenReceivedTxn_other,time_avgBetweenTxn_other,time_diffBetweenFirstAndLastSent_other,time_diffBetweenFirstAndLastTxn_other,erc20_uniqueTokenAddressesSent_other,erc20_uniqueTokenAddressesReceived_other,erc20_mostSentTokenAddress_other,erc20_mostReceivedTokenAddress_other,erc20_uniqueUnknownValueTokens_other,erc20_transactionsWithUnknownValueTokens_other,walletAddress,total_transactions_transfer,total_etherSent_transfer,total_etherReceived_transfer,total_etherSentToContract_transfer,total_sentTxn_transfer,total_receivedTxn_transfer,total_sentToContractTxn_transfer,total_uniqueReceivedFromAddresses_transfer,total_uniqueSentToAddresses_transfer,time_avgBetweenTxn_transfer,time_diffBetweenFirstAndLastTxn_transfer,erc20_uniqueTokenAddressesSent_transfer,erc20_uniqueTokenAddressesReceived_transfer,erc20_mostSentTokenAddress_transfer,erc20_uniqueUnknownValueTokens_transfer,erc20_transactionsWithUnknownValueTokens_transfer,total_transactions_approval,total_etherSent_approval,total_etherReceived_approval,total_etherSentToContract_approval,total_sentTxn_approval,total_receivedTxn_approval,total_sentToContractTxn_approval,total_uniqueReceivedFromAddresses_approval,total_uniqueSentToAddresses_approval,time_avgBetweenTxn_approval,time_diffBetweenFirstAndLastTxn_approval,erc20_uniqueTokenAddressesSent_approval,erc20_uniqueTokenAddressesReceived_approval,erc20_mostSentTokenAddress_approval,erc20_uniqueUnknownValueTokens_approval,erc20_transactionsWithUnknownValueTokens_approval,received:minValue,received:maxValue,received:avgValue,sent:minValue,sent:maxValue,sent:avgValue,sentToContract:minValue,sentToContract:maxValue,sentToContract:avgValue,total:transactions,total:etherSent,total:etherReceived,total:etherSentToContract,total:sentTxn,total:receivedTxn,total:sentToContractTxn,total:uniqueReceivedFromAddresses,total:uniqueSentToAddresses,time:avgBetweenSentTxn,time:avgBetweenReceivedTxn,time:avgBetweenTxn,time:diffBetweenFirstAndLastSent,time:diffBetweenFirstAndLastReceived,time:diffBetweenFirstAndLastTxn,currentTime
0,1,0,1,None,0 days 00:00:00,None,0 days 00:00:00,1,0,0xdac17f958d2ee523a2206206994597c13d831ec7,None,1,1,0xe1c63ef1a2f00840cba26d13249a247bc8ad40bf,1,0,0,0,0,0,0,0,1,0 days 00:00:00,0 days 00:00:00,1,0,0xdac17f958d2ee523a2206206994597c13d831ec7,1,1,2,0,0,0,0,0,0,0,1,0 days 05:22:00,0 days 10:44:00,1,0,0xdac17f958d2ee523a2206206994597c13d831ec7,1,2,0.001000,0.001000,0.001000,0.0,0.000000,0.000000,0.0,0.0,0.0,2,0.000000,0.001000,0.0,1,1,1,1,1,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,NaN
1,1,0,1,None,0 days 00:00:00,None,0 days 00:00:00,1,0,0xdac17f958d2ee523a2206206994597c13d831ec7,None,1,1,0x7a0c5bf3ac8778bd63efb914f89c048447331dab,8,0,0,0,0,0,0,0,6,0 days 02:05:21,0 days 16:42:48,1,0,0xdac17f958d2ee523a2206206994597c13d831ec7,1,8,1,0,0,0,0,0,0,0,1,0 days 00:00:00,0 days 00:00:00,1,0,0xdac17f958d2ee523a2206206994597c13d831ec7,1,1,0.000636,0.012194,0.003856,0.0,0.001724,0.000345,0.0,0.0,0.0,14,0.003447,0.015424,0.0,10,4,8,3,2,0 days 01:40:16.800000,0 days 04:57:57,0 days 01:11:37.714285714,0 days 16:42:48,0 days 19:51:48,0 days 16:42:48,NaN
2,2,0,1,None,0 days 00:07:18,None,0 days 00:14:36,1,0,0xdac17f958d2ee523a2206206994597c13d831ec7,None,1,2,0xe4af40b5b6a4c7ba9aacfe86a95f3da924648ac2,1,0,0,0,0,0,0,0,1,0 days 00:00:00,0 days 00:00:00,1,0,0xdac17f958d2ee523a2206206994597c13d831ec7,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,0.012382,0.012382,0.012382,0.0,0.000000,0.000000,0.0,0.0,0.0,3,0.000000,0.024764,0.0,1,2,1,1,1,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,NaN
3,10,0,1,None,0 days 03:06:00,None,1 days 07:00:00,1,0,0xdac17f958d2ee523a2206206994597c13d831ec7,None,1,10,0xaf60ac3178567ef1a8110c0cfc4e3df68c61f3e2,2,0,0,0,0,0,0,0,2,0 days 03:27:06,0 days 06:54:12,1,0,0xdac17f958d2ee523a2206206

In [35]:
erc20_abn_analytics.shape

(99, 71)

In [91]:
erc20_abn_analytics.to_csv('erc20_abn_analytics.csv')

# 2. Normal Wallets

## 2.1 Load Normal Wallets, Trx and ERC-20:

### 2.1.1 Normal Wallets

In [2]:
import glob
import pandas as pd

df_Nrm_wllts = pd.read_csv('./healthy_wallets.csv')
df_Nrm_wllts = df_Nrm_wllts[df_Nrm_wllts['is_contract']==False]
df_Nrm_wllts.head()

,Unnamed: 0.1,Unnamed: 0,healthy_wallets,Block,is_contract
5,5,5,0x0000000000C521824EaFf97Eac7B73B084ef9306,15407624.0,False
7,7,7,0x000000000D71b31F9C460f26C45589EC91551969,15511116.0,False
9,9,9,0x000000001a3494Ed0ee59623FFAA46995A6c99A9,15397834.0,False
14,14,14,0x0000208a6cC0299dA631C08fE8c2EDe435Ea83B8,15407624.0,False
16,16,16,0x0000Bf7f4e4B7fb2315fc6D5d0F8854c91dfF1d8,15407624.0,False


### 2.1.2 Normal Trx

In [3]:
df_Nrm_trx = pd.read_csv('./agg_healthy_trx.csv')
df_Nrm_trx['Trx_Datetime'] = pd.to_datetime(df_Nrm_trx['Trx_Datetime'])

In [4]:
df_Nrm_trx.shape

(2421774, 13)

### 2.1.3 Normal ERC-20

In [5]:
df_Nrm_erc20_appr = pd.concat(map(pd.read_csv, glob.glob('./Pre_feat_health_trx/agg_smartcontract_APPROVALS_*.csv')))
df_Nrm_erc20_appr['ERC20Info_eventInfo_transferReceiver'] = None
df_Nrm_erc20_appr['ERC20Info_eventInfo_transferReceiverIsContract'] = None

In [6]:
df_Nrm_erc20_appr.shape

(19767, 20)

In [7]:
df_Nrm_erc20_trnsf = pd.concat(map(pd.read_csv, glob.glob('./Pre_feat_health_trx/agg_smartcontract_TRANSFERS_*.csv')))

/tmp/ipykernel_6595/3111478084.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_Nrm_erc20_trnsf = pd.concat(map(pd.read_csv, glob.glob('./Pre_feat_health_trx/agg_smartcontract_TRANSFERS_*.csv')))


In [8]:
df_Nrm_erc20_trnsf = df_Nrm_erc20_trnsf.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'ERC20Info_eventInfo'])

In [9]:
df_Nrm_erc20_other = pd.concat(map(pd.read_csv, glob.glob('./Pre_feat_health_trx/agg_smartcontract_OTHER_*.csv')))

In [10]:
df_Nrm_erc20_other['ERC20Info_eventInfo_transferReceiver'] = df_Nrm_erc20_other['receiver']
df_Nrm_erc20_other['ERC20Info_eventInfo_transferReceiverIsContract'] = True
df_Nrm_erc20_other = df_Nrm_erc20_other.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'ERC20Info_eventInfo'])
#df_Nrm_erc20_other['ERC20Info_eventInfo_type']='nan'
#df_Nrm_erc20_other['ERC20Info_eventInfo_tokenQuantity']='nan'

In [11]:
df_Nrm_erc20_other.shape

(3163, 14)

## 2.2 Generate Eth Transactional Features

In [12]:
df_nrm_out_trx = pd.DataFrame(columns=['received:minValue', 'received:maxValue', 'received:avgValue',
       'sent:minValue', 'sent:maxValue', 'sent:avgValue',
       'sentToContract:minValue', 'sentToContract:maxValue',
       'sentToContract:avgValue', 'total:transactions', 'total:etherSent',
       'total:etherReceived', 'total:etherSentToContract', 'total:sentTxn',
       'total:receivedTxn', 'total:sentToContractTxn',
       'total:uniqueReceivedFromAddresses', 'total:uniqueSentToAddresses',
       'time:avgBetweenSentTxn', 'time:avgBetweenReceivedTxn',
       'time:avgBetweenTxn', 'time:diffBetweenFirstAndLastSent',
       'time:diffBetweenFirstAndLastReceived',
       'time:diffBetweenFirstAndLastTxn', 'walletAddress', 'currentTime'])

In [13]:
from pandas.tseries.offsets import DateOffset
from eth_feature_functions import get_processed_erc20_txn_list_analytics
from eth_feature_functions import get_processed_eth_txn_list_analytics
from eth_feature_functions import unflatten
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def dateoffset3Months(df, walletAddress):
    
    detected_date = df[((df['Trx_From_Wallet']==walletAddress) 
                        | (df['Trx_To_Wallet']==walletAddress))]['Trx_Datetime'].max()
    min_date = detected_date - DateOffset(months=3)
    
    return detected_date, min_date


# Create a list out of all the transaction hashes of transactions to and from the walletAddress
with tqdm(total=len(df_Nrm_wllts.healthy_wallets)) as pbar:
    for walletAddress in list(df_Nrm_wllts.healthy_wallets):
        
        detected_date, min_date = dateoffset3Months(df_Nrm_trx, walletAddress)

        df = df_Nrm_trx[((df_Nrm_trx['Trx_From_Wallet']==walletAddress) 
                        | (df_Nrm_trx['Trx_To_Wallet']==walletAddress))
                        & (df_Nrm_trx['Trx_Datetime'] >= min_date)
                        & (df_Nrm_trx['Trx_Datetime'] <= detected_date)]

        pbar.update(1)

        df = df.rename(columns = lambda x: x.replace('_', ''))

        flattenedDictList = df.values.tolist()
        ethList = []

        for values in flattenedDictList:
            d = dict(zip(df.keys(), values))
            ethList.append(unflatten(d, '_'))

        analytics = flatdict.FlatDict(get_processed_eth_txn_list_analytics(ethList, walletAddress))

        # Transform all values to lower case
        for key in analytics:
            if type(analytics[key]) == str:
                analytics[key] = analytics[key].lower()
            analytics[key] = [analytics[key]]

        df_nrm_out_trx = df_nrm_out_trx.append(pd.DataFrame(dict(analytics)), ignore_index=True)
        

100%|██████████| 9060/9060 [1:22:37<00:00,  1.83it/s]


In [14]:
#df_nrm_out_trx = df_nrm_out_trx.drop(columns=['currentTime'])
df_nrm_out_trx.shape

(4727, 26)

In [15]:
df_nrm_out_trx.to_csv('erc20_nrm_analytics.csv')

## 2.3 Generate ERC-20 Features

In [38]:
from pandas.tseries.offsets import DateOffset
from eth_feature_functions import get_processed_erc20_txn_list_analytics
from eth_feature_functions import get_processed_eth_txn_list_analytics
from eth_feature_functions import unflatten
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def dateoffset3Months(df, walletAddress):
    
    detected_date = df[((df['Trx_From_Wallet'].str.lower()==walletAddress) 
                        | (df['Trx_To_Wallet'].str.lower()==walletAddress))]['Trx_Datetime'].max()
    min_date = detected_date - DateOffset(months=3)
    
    return detected_date, min_date


def process_erc20_analytics(wllt_list, df_erc20, df_trx, erc20_type):
    
    df_out = pd.DataFrame(columns=['received:minValue', 'received:maxValue', 'received:avgValue',
       'sent:minValue', 'sent:maxValue', 'sent:avgValue', 'sentToContract:minValue', 'sentToContract:maxValue',
       'sentToContract:avgValue', 'total:transactions', 'total:etherSent', 'total:etherReceived', 'total:etherSentToContract', 'total:sentTxn',
       'total:receivedTxn', 'total:sentToContractTxn', 'total:uniqueReceivedFromAddresses', 'total:uniqueSentToAddresses',
       'time:avgBetweenSentTxn', 'time:avgBetweenReceivedTxn', 'time:avgBetweenTxn', 'time:diffBetweenFirstAndLastSent',
       'time:diffBetweenFirstAndLastReceived', 'time:diffBetweenFirstAndLastTxn', 'erc20:uniqueTokenAddressesSent',
       'erc20:uniqueTokenAddressesReceived', 'erc20:mostSentTokenAddress', 'erc20:mostReceivedTokenAddress', 'erc20:uniqueUnknownValueTokens',
       'erc20:transactionsWithUnknownValueTokens', 'walletAddress','currentTime'])
    
    with tqdm(total=len(wllt_list)) as pbar:
        for walletAddress in wllt_list:

            detected_date, min_date = dateoffset3Months(df_trx, walletAddress)
            
            pbar.update(1)
            df_ = df_erc20[((df_erc20['sender']==walletAddress) 
                      | (df_erc20['receiver']==walletAddress))
                     #& (df_erc20['ERC20Info_eventInfo_type']==erc20_type)
                     & (df_erc20['Trx_Datetime'] >= min_date)
                     & (df_erc20['Trx_Datetime'] <= detected_date)]
            
            #print(str(walletAddress) + '|' + str(min_date) + '|' + str(detected_date))
            #break
            if df_.empty:
                continue
            
            flattenedDictList = df_.values.tolist()
            ethList = []

            for values in flattenedDictList:
                d = dict(zip(df_.keys(), values))
                ethList.append(unflatten(d, '_'))

            analytics = flatdict.FlatDict(get_processed_erc20_txn_list_analytics(ethList, walletAddress))

            # Transform all values to lower case
            for key in analytics:
                if type(analytics[key]) == str:
                    analytics[key] = analytics[key].lower()
                analytics[key] = [analytics[key]]

            # Must be converted to dict from FlatDict
            df_out = df_out.append(pd.DataFrame(dict(analytics)), ignore_index=True)
            
    return df_out

### 2.3.1 Approvals

In [141]:
#timestamp to datetime
from datetime import datetime

df_Nrm_erc20_appr['Trx_Datetime'] = [datetime.fromtimestamp(x) for x in df_Nrm_erc20_appr['timestamp']]
df_Nrm_erc20_appr['timestamp'] = pd.to_datetime(df_Nrm_erc20_appr['timestamp'], unit='s')

In [ ]:
erc20_type = 'approval'
df_nrm_out_erc20_app = process_erc20_analytics(list(df_Nrm_wllts.healthy_wallets.str.lower()),
                                               df_Nrm_erc20_appr,
                                               df_Nrm_trx,
                                               erc20_type)

100%|██████████| 9060/9060 [3:23:03<00:00,  1.34s/it]  


In [151]:
df_nrm_out_erc20_app.shape

(773, 17)

In [146]:
df_nrm_out_erc20_app = df_nrm_out_erc20_app.drop(columns=['received:minValue','received:maxValue','received:avgValue',
                                                              'sent:minValue','sent:maxValue','sent:avgValue', 
                                                              'sentToContract:minValue','sentToContract:maxValue',
                                                              'sentToContract:avgValue', 'time:avgBetweenSentTxn',
                                                          'time:avgBetweenReceivedTxn', 'time:diffBetweenFirstAndLastSent',
                                                          'time:diffBetweenFirstAndLastReceived', 'erc20:mostReceivedTokenAddress',
                                                         'currentTime'])


In [147]:
df_nrm_out_erc20_app.columns = df_nrm_out_erc20_app.columns.str.replace(":", "_", regex=True)

In [149]:
df_nrm_out_erc20_app = df_nrm_out_erc20_app.rename(columns={c: c+'_approval' 
                                                            for c in df_nrm_out_erc20_app.columns 
                                                                if c not in ['walletAddress','currentTime']})

In [152]:
df_nrm_out_erc20_app.to_csv('erc20_nrm_approval_analytics.csv')

### 2.3.2 Transfers

In [44]:
#list(df_Nrm_wllts.healthy_wallets.str.lower())
df_Nrm_wllts_trnsf = df_Nrm_wllts[(df_Nrm_wllts['healthy_wallets'].str.lower().isin(list(df_Nrm_erc20_trnsf['receiver'])))
                                 | (df_Nrm_wllts['healthy_wallets'].str.lower().isin(list(df_Nrm_erc20_trnsf['sender'])))]
df_Nrm_wllts_trnsf.shape

(1147, 5)

In [45]:
df_Nrm_erc20_trnsf['Trx_Datetime'] = [datetime.fromtimestamp(x) for x in df_Nrm_erc20_trnsf['timestamp']]
df_Nrm_erc20_trnsf['timestamp'] = pd.to_datetime(df_Nrm_erc20_trnsf['timestamp'], unit='s')

In [ ]:
erc20_type = 'transfer'
df_nrm_out_erc20_transfer = process_erc20_analytics(list(df_Nrm_wllts_trnsf.healthy_wallets.str.lower()),
                                                    df_Nrm_erc20_trnsf,
                                                    df_Nrm_trx,
                                                    erc20_type)

 11%|█         | 121/1147 [02:51<23:41,  1.39s/it]

In [ ]:
df_nrm_out_erc20_transfer.head()

In [ ]:
df_nrm_out_erc20_transfer = df_nrm_out_erc20_transfer.drop(columns=['received:minValue','received:maxValue','received:avgValue',
                                                              'sent:minValue','sent:maxValue','sent:avgValue', 
                                                              'sentToContract:minValue','sentToContract:maxValue',
                                                              'sentToContract:avgValue', 'time:avgBetweenSentTxn',
                                                          'time:avgBetweenReceivedTxn', 'time:diffBetweenFirstAndLastSent',
                                                          'time:diffBetweenFirstAndLastReceived', 'erc20:mostReceivedTokenAddress',
                                                                   'currentTime'])


In [ ]:
df_nrm_out_erc20_transfer.columns = df_nrm_out_erc20_transfer.columns.str.replace(":", "_", regex=True)

In [ ]:
df_nrm_out_erc20_transfer = df_nrm_out_erc20_transfer.rename(columns={c: c+'_transfer' 
                                                            for c in df_nrm_out_erc20_transfer.columns 
                                                                if c not in ['walletAddress','currentTime']})

In [ ]:
df_nrm_out_erc20_transfer.to_csv('erc20_nrm_transfer_analytics.csv')

### 2.3.3 Other

In [ ]:
#list(df_Nrm_wllts.healthy_wallets.str.lower())
df_Nrm_wllts_othr = df_Nrm_wllts[(df_Nrm_wllts['healthy_wallets'].str.lower().isin(list(df_Nrm_erc20_other['receiver'])))
                                 | (df_Nrm_wllts['healthy_wallets'].str.lower().isin(list(df_Nrm_erc20_other['sender'])))]
df_Nrm_wllts_othr.shape

In [27]:
df_Nrm_wllts_othr.head()

,Unnamed: 0.1,Unnamed: 0,healthy_wallets,Block,is_contract
2156,2156,2156,0x3D56C1734FaB2126f3A6a58bcf57C25B1e99372B,15296505.0,False
2159,2159,2159,0x3D88A762cc6E039840a10656DBC0f9d3D1b31432,15035173.0,False
2164,2164,2164,0x3DA2E7eD450f747cFAAc81c3d32605Beb83de70C,14613635.0,False
2167,2167,2167,0x3DE56d8f088b6767557dDe6eb2d83bF74DF6BEc3,15407624.0,False
2169,2169,2169,0x3Dadf855b9E30e63Bee10ec4D92cbFE789E5F8A8,15898566.0,False


In [17]:
df_Nrm_erc20_other['Trx_Datetime'] = [datetime.fromtimestamp(x) for x in df_Nrm_erc20_other['timestamp']]
df_Nrm_erc20_other['timestamp'] = pd.to_datetime(df_Nrm_erc20_other['timestamp'], unit='s')

In [ ]:
erc20_type = 'other'
df_nrm_out_erc20_other = process_erc20_analytics(list(df_Nrm_wllts_othr.healthy_wallets.str.lower()), 
                                                 df_Nrm_erc20_other,
                                                 df_Nrm_trx,  
                                                 erc20_type)

100%|██████████| 291/291 [06:41<00:00,  1.38s/it]


In [33]:
df_nrm_out_erc20_other.head()

,total_transactions_other,total_uniqueReceivedFromAddresses_other,total_uniqueSentToAddresses_other,time_avgBetweenReceivedTxn_other,time_avgBetweenTxn_other,time_diffBetweenFirstAndLastSent_other,time_diffBetweenFirstAndLastTxn_other,erc20_uniqueTokenAddressesSent_other,erc20_uniqueTokenAddressesReceived_other,erc20_mostSentTokenAddress_other,erc20_mostReceivedTokenAddress_other,erc20_uniqueUnknownValueTokens_other,erc20_transactionsWithUnknownValueTokens_other,walletAddress
0,5,0,1,None,14 days 00:10:46.400000,None,70 days 00:53:52,1,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,None,1,5,0x3d56c1734fab2126f3a6a58bcf57c25b1e99372b
1,1,0,1,None,0 days 00:00:00,None,0 days 00:00:00,1,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,None,1,1,0x3da2e7ed450f747cfaac81c3d32605beb83de70c
2,2,0,1,None,2 days 00:20:46,None,4 days 00:41:32,1,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,None,1,2,0x3de56d8f088b6767557dde6eb2d83bf74df6bec3
3,12,0,1,None,6 days 17:09:23.166666666,None,80 days 13:52:38,1,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,None,1,12,0x3dadf855b9e30e63bee10ec4d92cbfe789e5f8a8
4,2,0,1,None,1 days 09:13:26,None,2 days 18:26:52,1,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,None,1,2,0x3dd387da5b32160f3f4c8fb2ec0adb415cad41f1


In [34]:
df_nrm_out_erc20_other = df_nrm_out_erc20_other.drop(columns=['received:minValue','received:maxValue','received:avgValue',
                                                              'sent:minValue','sent:maxValue','sent:avgValue', 
                                                              'sentToContract:minValue','sentToContract:maxValue',
                                                              'sentToContract:avgValue', 'time:avgBetweenSentTxn',
                                                              'time:diffBetweenFirstAndLastReceived', 'total:etherSent',
                                                              'total:etherReceived','total:etherSentToContract',
                                                              'total:sentTxn','total:receivedTxn','total:sentToContractTxn',
                                                             'currentTime'])


KeyError: "['received:minValue', 'received:maxValue', 'received:avgValue', 'sent:minValue', 'sent:maxValue', 'sent:avgValue', 'sentToContract:minValue', 'sentToContract:maxValue', 'sentToContract:avgValue', 'time:avgBetweenSentTxn', 'time:diffBetweenFirstAndLastReceived', 'total:etherSent', 'total:etherReceived', 'total:etherSentToContract', 'total:sentTxn', 'total:receivedTxn', 'total:sentToContractTxn', 'currentTime'] not found in axis"

In [ ]:
df_nrm_out_erc20_other.columns = df_nrm_out_erc20_other.columns.str.replace(":", "_", regex=True)

In [ ]:
df_nrm_out_erc20_other = df_nrm_out_erc20_other.rename(columns={c: c+'_other' 
                                                            for c in df_nrm_out_erc20_other.columns 
                                                                if c not in ['walletAddress','currentTime']})

In [ ]:
df_nrm_out_erc20_other.to_csv('erc20_nrm_other_analytics.csv')

## 2.3 Agg Normal analytics

In [114]:
df_nrm_out_erc20_other = pd.read_csv('./erc20_nrm_other_analytics.csv')
df_nrm_out_erc20_appr = pd.read_csv('./erc20_nrm_approval_analytics.csv')
df_nrm_out_erc20_trnsf = pd.read_csv('./erc20_nrm_transfer_analytics.csv')

In [115]:
df_nrm_out_trx = pd.read_csv('./erc20_nrm_analytics.csv')

In [116]:
erc20_nrm_analytics = df_nrm_out_erc20_other.merge(df_nrm_out_erc20_appr,
                                                   left_on='walletAddress',
                                                   right_on='walletAddress', how='outer')
erc20_nrm_analytics = erc20_nrm_analytics.merge(df_nrm_out_erc20_trnsf,
                                                left_on='walletAddress',
                                                right_on='walletAddress', how='outer')

In [117]:
erc20_nrm_analytics = erc20_nrm_analytics.merge(df_nrm_out_trx,
                                                left_on='walletAddress',
                                                right_on='walletAddress', how='outer')

In [118]:
erc20_nrm_analytics.shape

(4727, 75)

# 3. Define Target and create DS!

In [119]:
erc20_abn_analytics = pd.read_csv('./erc20_abn_analytics.csv')

In [120]:
erc20_abn_analytics.head()

,Unnamed: 0,total_transactions_other,total_uniqueReceivedFromAddresses_other,total_uniqueSentToAddresses_other,time_avgBetweenReceivedTxn_other,time_avgBetweenTxn_other,time_diffBetweenFirstAndLastSent_other,time_diffBetweenFirstAndLastTxn_other,erc20_uniqueTokenAddressesSent_other,erc20_uniqueTokenAddressesReceived_other,erc20_mostSentTokenAddress_other,erc20_mostReceivedTokenAddress_other,erc20_uniqueUnknownValueTokens_other,erc20_transactionsWithUnknownValueTokens_other,walletAddress,total_transactions_transfer,total_etherSent_transfer,total_etherReceived_transfer,total_etherSentToContract_transfer,total_sentTxn_transfer,total_receivedTxn_transfer,total_sentToContractTxn_transfer,total_uniqueReceivedFromAddresses_transfer,total_uniqueSentToAddresses_transfer,time_avgBetweenTxn_transfer,time_diffBetweenFirstAndLastTxn_transfer,erc20_uniqueTokenAddressesSent_transfer,erc20_uniqueTokenAddressesReceived_transfer,erc20_mostSentTokenAddress_transfer,erc20_uniqueUnknownValueTokens_transfer,erc20_transactionsWithUnknownValueTokens_transfer,total_transactions_approval,total_etherSent_approval,total_etherReceived_approval,total_etherSentToContract_approval,total_sentTxn_approval,total_receivedTxn_approval,total_sentToContractTxn_approval,total_uniqueReceivedFromAddresses_approval,total_uniqueSentToAddresses_approval,time_avgBetweenTxn_approval,time_diffBetweenFirstAndLastTxn_approval,erc20_uniqueTokenAddressesSent_approval,erc20_uniqueTokenAddressesReceived_approval,erc20_mostSentTokenAddress_approval,erc20_uniqueUnknownValueTokens_approval,erc20_transactionsWithUnknownValueTokens_approval,received:minValue,received:maxValue,received:avgValue,sent:minValue,sent:maxValue,sent:avgValue,sentToContract:minValue,sentToContract:maxValue,sentToContract:avgValue,total:transactions,total:etherSent,total:etherReceived,total:etherSentToContract,total:sentTxn,total:receivedTxn,total:sentToContractTxn,total:uniqueReceivedFromAddresses,total:uniqueSentToAddresses,time:avgBetweenSentTxn,time:avgBetweenReceivedTxn,time:avgBetweenTxn,time:diffBetweenFirstAndLastSent,time:diffBetweenFirstAndLastReceived,time:diffBetweenFirstAndLastTxn
0,0,1.0,0.0,1.0,NaN,0 days 00:00:00,NaN,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,NaN,1.0,1.0,0xe1c63ef1a2f00840cba26d13249a247bc8ad40bf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 00:00:00,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 05:22:00,0 days 10:44:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,2.0,0.001000,0.001000,0.001000,0.0,0.000000,0.000000,0.0,0.0,0.0,2.0,0.000000,0.001000,0.0,1.0,1.0,1.0,1.0,1.0,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00
1,1,1.0,0.0,1.0,NaN,0 days 00:00:00,NaN,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,NaN,1.0,1.0,0xe1c63ef1a2f00840cba26d13249a247bc8ad40bf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 00:00:00,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 05:22:00,0 days 10:44:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,2.0,0.001000,0.001000,0.001000,0.0,0.000000,0.000000,0.0,0.0,0.0,2.0,0.000000,0.001000,0.0,1.0,1.0,1.0,1.0,1.0,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00
2,2,1.0,0.0,1.0,NaN,0 days 00:00:00,NaN,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,NaN,1.0,1.0,0x7a0c5bf3ac8778bd63efb914f89c048447331dab,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0 days 02:05:21,0 days 16:42:48,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 00:00:00,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,1.0,0.000636,0.012194,0.003856,0.0,0.001724,0.000345,0.0,0.0,0.0,14.0,0.003447,0.015424,0.0,10.0,4.0,8.0,3.0,2.0,0 days 01:40:16.800000,0 days 04:57:57,0 days 01:11:37.714285714,0 days 16:42:48,0 days 1

## 3.1 Define abnormal target=1

In [121]:
erc20_abn_analytics['Target'] = 1

In [122]:
erc20_abn_analytics = erc20_abn_analytics.drop(columns={'Unnamed: 0',})

In [123]:
erc20_abn_analytics.head()

,total_transactions_other,total_uniqueReceivedFromAddresses_other,total_uniqueSentToAddresses_other,time_avgBetweenReceivedTxn_other,time_avgBetweenTxn_other,time_diffBetweenFirstAndLastSent_other,time_diffBetweenFirstAndLastTxn_other,erc20_uniqueTokenAddressesSent_other,erc20_uniqueTokenAddressesReceived_other,erc20_mostSentTokenAddress_other,erc20_mostReceivedTokenAddress_other,erc20_uniqueUnknownValueTokens_other,erc20_transactionsWithUnknownValueTokens_other,walletAddress,total_transactions_transfer,total_etherSent_transfer,total_etherReceived_transfer,total_etherSentToContract_transfer,total_sentTxn_transfer,total_receivedTxn_transfer,total_sentToContractTxn_transfer,total_uniqueReceivedFromAddresses_transfer,total_uniqueSentToAddresses_transfer,time_avgBetweenTxn_transfer,time_diffBetweenFirstAndLastTxn_transfer,erc20_uniqueTokenAddressesSent_transfer,erc20_uniqueTokenAddressesReceived_transfer,erc20_mostSentTokenAddress_transfer,erc20_uniqueUnknownValueTokens_transfer,erc20_transactionsWithUnknownValueTokens_transfer,total_transactions_approval,total_etherSent_approval,total_etherReceived_approval,total_etherSentToContract_approval,total_sentTxn_approval,total_receivedTxn_approval,total_sentToContractTxn_approval,total_uniqueReceivedFromAddresses_approval,total_uniqueSentToAddresses_approval,time_avgBetweenTxn_approval,time_diffBetweenFirstAndLastTxn_approval,erc20_uniqueTokenAddressesSent_approval,erc20_uniqueTokenAddressesReceived_approval,erc20_mostSentTokenAddress_approval,erc20_uniqueUnknownValueTokens_approval,erc20_transactionsWithUnknownValueTokens_approval,received:minValue,received:maxValue,received:avgValue,sent:minValue,sent:maxValue,sent:avgValue,sentToContract:minValue,sentToContract:maxValue,sentToContract:avgValue,total:transactions,total:etherSent,total:etherReceived,total:etherSentToContract,total:sentTxn,total:receivedTxn,total:sentToContractTxn,total:uniqueReceivedFromAddresses,total:uniqueSentToAddresses,time:avgBetweenSentTxn,time:avgBetweenReceivedTxn,time:avgBetweenTxn,time:diffBetweenFirstAndLastSent,time:diffBetweenFirstAndLastReceived,time:diffBetweenFirstAndLastTxn,Target
0,1.0,0.0,1.0,NaN,0 days 00:00:00,NaN,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,NaN,1.0,1.0,0xe1c63ef1a2f00840cba26d13249a247bc8ad40bf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 00:00:00,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 05:22:00,0 days 10:44:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,2.0,0.001000,0.001000,0.001000,0.0,0.000000,0.000000,0.0,0.0,0.0,2.0,0.000000,0.001000,0.0,1.0,1.0,1.0,1.0,1.0,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,1
1,1.0,0.0,1.0,NaN,0 days 00:00:00,NaN,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,NaN,1.0,1.0,0xe1c63ef1a2f00840cba26d13249a247bc8ad40bf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 00:00:00,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 05:22:00,0 days 10:44:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,2.0,0.001000,0.001000,0.001000,0.0,0.000000,0.000000,0.0,0.0,0.0,2.0,0.000000,0.001000,0.0,1.0,1.0,1.0,1.0,1.0,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,1
2,1.0,0.0,1.0,NaN,0 days 00:00:00,NaN,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,NaN,1.0,1.0,0x7a0c5bf3ac8778bd63efb914f89c048447331dab,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0 days 02:05:21,0 days 16:42:48,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 00:00:00,0 days 00:00:00,1.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,1.0,1.0,0.000636,0.012194,0.003856,0.0,0.001724,0.000345,0.0,0.0,0.0,14.0,0.003447,0.015424,0.0,10.0,4.0,8.0,3.0,2.0,0 days 01:40:16.800000,0 days 04:57:57,0 days 01:11:37.714285714,0 days 16:42:48,0 days 19:51:4

In [124]:
erc20_abn_analytics.shape

(187, 71)

## 3.2 Define Normal target = 0

In [125]:
erc20_nrm_analytics['Target'] = 0
erc20_nrm_analytics.head()

,Unnamed: 0_x,total_transactions_other,total_uniqueReceivedFromAddresses_other,total_uniqueSentToAddresses_other,time_avgBetweenReceivedTxn_other,time_avgBetweenTxn_other,time_diffBetweenFirstAndLastSent_other,time_diffBetweenFirstAndLastTxn_other,erc20_uniqueTokenAddressesSent_other,erc20_uniqueTokenAddressesReceived_other,erc20_mostSentTokenAddress_other,erc20_mostReceivedTokenAddress_other,erc20_uniqueUnknownValueTokens_other,erc20_transactionsWithUnknownValueTokens_other,walletAddress,Unnamed: 0_y,total_transactions_approval,total_etherSent_approval,total_etherReceived_approval,total_etherSentToContract_approval,total_sentTxn_approval,total_receivedTxn_approval,total_sentToContractTxn_approval,total_uniqueReceivedFromAddresses_approval,total_uniqueSentToAddresses_approval,time_avgBetweenTxn_approval,time_diffBetweenFirstAndLastTxn_approval,erc20_uniqueTokenAddressesSent_approval,erc20_uniqueTokenAddressesReceived_approval,erc20_mostSentTokenAddress_approval,erc20_uniqueUnknownValueTokens_approval,erc20_transactionsWithUnknownValueTokens_approval,Unnamed: 0_x,total_transactions_transfer,total_etherSent_transfer,total_etherReceived_transfer,total_etherSentToContract_transfer,total_sentTxn_transfer,total_receivedTxn_transfer,total_sentToContractTxn_transfer,total_uniqueReceivedFromAddresses_transfer,total_uniqueSentToAddresses_transfer,time_avgBetweenTxn_transfer,time_diffBetweenFirstAndLastTxn_transfer,erc20_uniqueTokenAddressesSent_transfer,erc20_uniqueTokenAddressesReceived_transfer,erc20_mostSentTokenAddress_transfer,erc20_uniqueUnknownValueTokens_transfer,erc20_transactionsWithUnknownValueTokens_transfer,Unnamed: 0_y,received:minValue,received:maxValue,received:avgValue,sent:minValue,sent:maxValue,sent:avgValue,sentToContract:minValue,sentToContract:maxValue,sentToContract:avgValue,total:transactions,total:etherSent,total:etherReceived,total:etherSentToContract,total:sentTxn,total:receivedTxn,total:sentToContractTxn,total:uniqueReceivedFromAddresses,total:uniqueSentToAddresses,time:avgBetweenSentTxn,time:avgBetweenReceivedTxn,time:avgBetweenTxn,time:diffBetweenFirstAndLastSent,time:diffBetweenFirstAndLastReceived,time:diffBetweenFirstAndLastTxn,currentTime,Target
0,0.0,5.0,0.0,1.0,NaN,14 days 00:10:46.400000,NaN,70 days 00:53:52,1.0,0.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaN,1.0,5.0,0x3d56c1734fab2126f3a6a58bcf57c25b1e99372b,442.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4 days 00:26:06.500000,8 days 00:52:13,2.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,2.0,2.0,541.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3 days 20:43:04.954545454,84 days 23:47:49,3.0,0.0,0x8355dbe8b0e275abad27eb843f3eaf3fc855e525,3.0,22.0,1779,0.010000,1.560714,0.330983,0.0,1.572896,0.049766,0.0,0.0,0.0,173,8.310931,1.985900,0.0,167,6,106,3,40,0 days 13:01:21.556886227,11 days 20:08:57.500000,0 days 12:34:15.606936416,90 days 14:47:00,71 days 00:53:45,90 days 14:47:00,NaN,0
1,1.0,1.0,0.0,1.0,NaN,0 days 00:00:00,NaN,0 days 00:00:00,1.0,0.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaN,1.0,1.0,0x3da2e7ed450f747cfaac81c3d32605beb83de70c,446.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 00:00:00,0 days 00:00:00,1.0,0.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1787,0.000000,0.337500,0.041197,0.0,0.319000,0.018373,0.0,0.0,0.0,63,0.863536,0.659159,0.0,47,16,27,4,21,1 days 19:26:18.489361702,5 days 01:27:07.250000,1 days 08:24:23.317460317,85 days 01:36:29,80 days 23:13:56,85 days 01:36:29,NaN,0
2,2.0,2.0,0.0,1.0,NaN,2 days 00:20:46,NaN,4 days 00:41:32,1.0,0.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaN,1.0,2.0,0x3de56d8f088b6767557dde6eb2d83bf74df6bec3,447.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2 days 20:49:01.333333333,17 days 04:54:08,5.0,0.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,5.0,6.0,544.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0 days 22:25:44,7 days 11:25:52,2.0,0.0,0xce3f08e664693ca792cace4af1364d5e220827b2,2.0,8.0,1790,0.000000,1.107007,0.447358,0.0,1.860855,0.155

In [126]:
erc20_nrm_analytics = erc20_nrm_analytics.drop(columns={'Unnamed: 0_x', 'Unnamed: 0_y', 'currentTime'})

In [127]:
erc20_nrm_analytics.shape

(4727, 71)

## 3.3 Append all info - FINAL DATASET

In [128]:
df = erc20_nrm_analytics.append(erc20_abn_analytics)

In [129]:
df.shape

(4914, 71)

In [130]:
df.head()

,total_transactions_other,total_uniqueReceivedFromAddresses_other,total_uniqueSentToAddresses_other,time_avgBetweenReceivedTxn_other,time_avgBetweenTxn_other,time_diffBetweenFirstAndLastSent_other,time_diffBetweenFirstAndLastTxn_other,erc20_uniqueTokenAddressesSent_other,erc20_uniqueTokenAddressesReceived_other,erc20_mostSentTokenAddress_other,erc20_mostReceivedTokenAddress_other,erc20_uniqueUnknownValueTokens_other,erc20_transactionsWithUnknownValueTokens_other,walletAddress,total_transactions_approval,total_etherSent_approval,total_etherReceived_approval,total_etherSentToContract_approval,total_sentTxn_approval,total_receivedTxn_approval,total_sentToContractTxn_approval,total_uniqueReceivedFromAddresses_approval,total_uniqueSentToAddresses_approval,time_avgBetweenTxn_approval,time_diffBetweenFirstAndLastTxn_approval,erc20_uniqueTokenAddressesSent_approval,erc20_uniqueTokenAddressesReceived_approval,erc20_mostSentTokenAddress_approval,erc20_uniqueUnknownValueTokens_approval,erc20_transactionsWithUnknownValueTokens_approval,total_transactions_transfer,total_etherSent_transfer,total_etherReceived_transfer,total_etherSentToContract_transfer,total_sentTxn_transfer,total_receivedTxn_transfer,total_sentToContractTxn_transfer,total_uniqueReceivedFromAddresses_transfer,total_uniqueSentToAddresses_transfer,time_avgBetweenTxn_transfer,time_diffBetweenFirstAndLastTxn_transfer,erc20_uniqueTokenAddressesSent_transfer,erc20_uniqueTokenAddressesReceived_transfer,erc20_mostSentTokenAddress_transfer,erc20_uniqueUnknownValueTokens_transfer,erc20_transactionsWithUnknownValueTokens_transfer,received:minValue,received:maxValue,received:avgValue,sent:minValue,sent:maxValue,sent:avgValue,sentToContract:minValue,sentToContract:maxValue,sentToContract:avgValue,total:transactions,total:etherSent,total:etherReceived,total:etherSentToContract,total:sentTxn,total:receivedTxn,total:sentToContractTxn,total:uniqueReceivedFromAddresses,total:uniqueSentToAddresses,time:avgBetweenSentTxn,time:avgBetweenReceivedTxn,time:avgBetweenTxn,time:diffBetweenFirstAndLastSent,time:diffBetweenFirstAndLastReceived,time:diffBetweenFirstAndLastTxn,Target
0,5.0,0.0,1.0,NaN,14 days 00:10:46.400000,NaN,70 days 00:53:52,1.0,0.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaN,1.0,5.0,0x3d56c1734fab2126f3a6a58bcf57c25b1e99372b,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4 days 00:26:06.500000,8 days 00:52:13,2.0,0.0,0xdac17f958d2ee523a2206206994597c13d831ec7,2.0,2.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3 days 20:43:04.954545454,84 days 23:47:49,3.0,0.0,0x8355dbe8b0e275abad27eb843f3eaf3fc855e525,3.0,22.0,0.010000,1.560714,0.330983,0.0,1.572896,0.049766,0.0,0.0,0.0,173.0,8.310931,1.985900,0.0,167.0,6.0,106.0,3.0,40.0,0 days 13:01:21.556886227,11 days 20:08:57.500000,0 days 12:34:15.606936416,90 days 14:47:00,71 days 00:53:45,90 days 14:47:00,0
1,1.0,0.0,1.0,NaN,0 days 00:00:00,NaN,0 days 00:00:00,1.0,0.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaN,1.0,1.0,0x3da2e7ed450f747cfaac81c3d32605beb83de70c,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0 days 00:00:00,0 days 00:00:00,1.0,0.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.337500,0.041197,0.0,0.319000,0.018373,0.0,0.0,0.0,63.0,0.863536,0.659159,0.0,47.0,16.0,27.0,4.0,21.0,1 days 19:26:18.489361702,5 days 01:27:07.250000,1 days 08:24:23.317460317,85 days 01:36:29,80 days 23:13:56,85 days 01:36:29,0
2,2.0,0.0,1.0,NaN,2 days 00:20:46,NaN,4 days 00:41:32,1.0,0.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaN,1.0,2.0,0x3de56d8f088b6767557dde6eb2d83bf74df6bec3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2 days 20:49:01.333333333,17 days 04:54:08,5.0,0.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,5.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0 days 22:25:44,7 days 11:25:52,2.0,0.0,0xce3f08e664693ca792cace4af1364d5e220827b2,2.0,8.0,0.000000,1.107007,0.447358,0.0,1.860855,0.155038,0.0,0.0,0.0,87.0,10.852685,7.605094,0.0,70.0,17.0,35.0,8.0,19.0,0 days 09:21:26.514285714,4 days 04:20:02

In [131]:
df.to_csv('./dataset.csv')

In [132]:
df['total:transactions'].sum()

898310.0